In [1]:
import math
import mindspore
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor, Parameter

TextRNN Model:

In [2]:
def make_batch(sentences, word_dict, n_class):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()  # space tokenizer
        input = [word_dict[n] for n in word[:-1]]  # create (1~n-1) as input
        target = word_dict[word[-1]]  # create (n) as target, We usually call this 'casual language model'

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [3]:
class TextRNN(nn.Cell):
    def __init__(self, n_class, n_hidden, batch_size):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, batch_first=True)
        self.W = nn.Dense(n_hidden, n_class, has_bias=False)
        self.b = Parameter(Tensor(np.ones([n_class]), mindspore.float32), 'b')

    def construct(self, X):
        X = X.swapaxes(0, 1) # X : [n_step, batch_size, n_class]
        outputs, _ = self.rnn(X)
        # outputs : [n_step, batch_size, num_directions(=1) * n_hidden]
        outputs = outputs[-1] # [batch_size, num_directions(=1) * n_hidden]
        model = self.W(outputs)# model : [batch_size, n_class]
        
        return model

In [4]:
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)
batch_size = len(sentences)

In [5]:
model = TextRNN(n_class, n_hidden, batch_size)

In [6]:
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [7]:
input_batch, target_batch = make_batch(sentences, word_dict, n_class)
input_batch = Tensor(input_batch, mindspore.float32)
target_batch = Tensor(target_batch, mindspore.int32)

In [8]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE)

net_with_criterion = nn.WithLossCell(model, criterion)
train_network = nn.TrainOneStepCell(net_with_criterion, optimizer)
train_network.set_train()

# Training
for epoch in range(5000):
    # hidden : [num_layers * num_directions, batch, hidden_size]
    loss = train_network(input_batch, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss.asnumpy()))

[WARNING] DEBUG(15818,0x10e7b9600,python):2022-01-23-19:53:38.027.011 [mindspore/ccsrc/debug/debugger/debugger.cc:95] Debugger] Not enabling debugger. Debugger does not support CPU.


Epoch: 1000 cost = 0.525997
Epoch: 2000 cost = 0.076064
Epoch: 3000 cost = 0.012782
Epoch: 4000 cost = 0.004983
Epoch: 5000 cost = 0.002537


In [9]:
# Predict
predict = model(input_batch).asnumpy().argmax(1)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
